In [21]:
# where is data?
#output_folder = '/home/jnaiman/data/morgan/' # HAL
output_folder = '/Users/jnaiman/Downloads/tmp/ocrpost/data/morgan/' # local

ender = '_small_words' # small has 100,000 for training, 5000 for dev

# model save dir
#model_save_dir = '/home/jnaiman/data/morgan/models/'
model_save_dir = '/Users/jnaiman/Downloads/tmp/ocrpost/data/morgan/models/' # local
model_file = 'new_torch_file_new_pages_small_words.pt'

file_dir = '/Users/jnaiman/Dropbox/wwt_image_extraction/OCRPostCorrection/alignments/'
test_file = 'test_masked_n5000_20230510.csv'


# its not 100% clear if we need this... setting a flag, but looks like we DO need it for memory issues
use_train_dev_size = True
train_size = 1000000
dev_size = 1000000

In [19]:
# char2i = pickle.load(open(output_folder + "data/char2i_new_pages.pkl", "rb"))
# i2char = pickle.load(open(output_folder + "data/i2char_new_pages.pkl", "rb"))
char2i = pickle.load(open(output_folder + "data/char2i_new_pages"+ender+".pkl", "rb"))
i2char = pickle.load(open(output_folder + "data/i2char_new_pages"+ender+".pkl", "rb"))

In [11]:
# #source = [list("abcdefghijkl"), list("mnopqrstwxyz")]
# #target = [list("ABCDEFGHIJKL"), list("MNOPQRSTWXYZ")] #update to astronomy vocab
# source = []#yy
# for s in model.source_index:
#     sentence = ""
#     for c in s:
#         #nu = source_index[c]
#         #sentence.append(c)
#         #sentence.append(num)
#         sentence += c
#     source.append(sentence)

In [13]:
import os
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
from nltk.lm import Vocabulary
import pickle
import sys
import torch
import importlib
from timeit import default_timer as t
sys.path.append("../")
#from metrics import levenshtein
import post_ocr_correction
from pytorch_beam_search import seq2seq

In [29]:
import pandas as pd
#!pip install Levenshtein
from Levenshtein import distance
import Levenshtein 

import re
from tqdm.auto import tqdm

In [24]:
from sys import path
path.append('../scienceDigitization/text_mining_ocr_and_pdf/ocr_spell_check/')
from importlib import reload
import utils
reload(utils)
from utils import align_texts_fast, get_fill_in_types

In [20]:
import torch

# model = Transformer(char2i, 
#                     i2char, 
#                     max_sequence_length = 110,
#                     embedding_dimension = 512, #256,
#                     feedforward_dimension = 2048, #1024,
#                     attention_heads = 8,
#                     encoder_layers = 4,
#                     decoder_layers = 4)
#                    #dropout = .5)
# print("model created")
# model.to(device)

# model = Transformer(char2i, 
#                     i2char, 
#                     max_sequence_length = 110,
#                     embedding_dimension = 512, #256,
#                     feedforward_dimension = 2048, #1024,
#                     attention_heads = 8,
#                     encoder_layers = 4,
#                     decoder_layers = 4)


model = Transformer(char2i, 
                    i2char, 
                    max_sequence_length = 110,
                    embedding_dimension = 512,
                    feedforward_dimension = 2048,
                    attention_heads = 8,
                    encoder_layers = 4,
                    decoder_layers = 4)#,
                   #dropout = .5)

model.load_state_dict(torch.load(model_save_dir + model_file, map_location=torch.device('cpu')))
#model = torch.load("../../data/torch_file60.pt", map_location=torch.device('cpu'))

Model: Seq2Seq Transformer
Source index: {'\t': 3, '\n': 4, ' ': 5, '!': 6, '"': 7, '#': 8, '$': 9, '%': 10, '&': 11, "'": 12, '(': 13, ')': 14, '*': 15, '+': 16, ',': 17, '-': 18, '.': 19, '/': 20, '0': 21, '1': 22, '2': 23, '3': 24, '4': 25, '5': 26, '6': 27, '7': 28, '8': 29, '9': 30, ':': 31, ';': 32, '<': 33, '<UNK>': 34, '=': 35, '>': 36, '?': 37, '@': 38, 'A': 39, 'B': 40, 'C': 41, 'D': 42, 'E': 43, 'F': 44, 'G': 45, 'H': 46, 'I': 47, 'J': 48, 'K': 49, 'L': 50, 'M': 51, 'N': 52, 'O': 53, 'P': 54, 'Q': 55, 'R': 56, 'S': 57, 'T': 58, 'U': 59, 'V': 60, 'W': 61, 'X': 62, 'Y': 63, 'Z': 64, '[': 65, '\\': 66, ']': 67, '_': 68, '`': 69, 'a': 70, 'b': 71, 'c': 72, 'd': 73, 'e': 74, 'f': 75, 'g': 76, 'h': 77, 'i': 78, 'j': 79, 'k': 80, 'l': 81, 'm': 82, 'n': 83, 'o': 84, 'p': 85, 'q': 86, 'r': 87, 's': 88, 't': 89, 'u': 90, 'v': 91, 'w': 92, 'x': 93, 'y': 94, 'z': 95, '{': 96, '|': 97, '}': 98, '~': 99, '\xa0': 100, '¡': 101, '¢': 102, '£': 103, '¥': 104, '§': 105, '©': 106, 'ª': 107, '«

<All keys matched successfully>

In [ ]:
# test_data = pd.read_csv('./source_dataframe.csv')
# print(test_data)

In [50]:
test_data = pd.read_csv(file_dir + test_file)

# start
ntest = 1

test_data = test_data.iloc[:ntest]

In [51]:
source = []
target = []
source_aligned = []
target_aligned = []
for i in range(len(test_data)):
    d = test_data.iloc[i]
    s = np.array(list(d['aligned sentences source'])) # aligned source
    t = np.array(list(d['aligned sentences target'])) # aligned target
    a = np.array(list(get_fill_in_types(d['aligned sentences target types'])))
    ss = "".join(s[np.where( (a == ' ') | (a == 'W') | (a == 'w'))[0]].tolist())
    tt = "".join(t[np.where( (a == ' ') | (a == 'W') | (a == 'w'))[0]].tolist())
    source_aligned.append(ss)
    target_aligned.append(tt)
    source.append(ss.replace('@',''))
    target.append(tt.replace('^',''))
    
test_data['words source aligned'] = source_aligned
test_data['words target aligned'] = target_aligned
test_data['words source'] = source
test_data['words target'] = target

In [52]:
mask = (pd.isnull(test_data['words source'])) | (pd.isnull(test_data['words target']))

test_data = test_data[~mask]

# now align
gs_aligned = []; ocr_aligned = []
gs = []; ocr = []
for o,p in zip(test_data['words source'].values, test_data['words target'].values):
    eops = Levenshtein.editops(o, p)
    ocr_text_aligned, pdf_text_aligned = align_texts_fast(o,p,eops)
    gs_aligned.append(pdf_text_aligned)
    # futze with ocr aligned so it matches data input
    ocr_text_aligned = ocr_text_aligned.replace('^', '@')
    ocr_aligned.append(ocr_text_aligned)
    gs.append(pdf_text_aligned.replace('@',''))
    ocr.append(ocr_text_aligned.replace('@',''))
    
test_data['gs_aligned'] = gs_aligned
test_data['ocr_aligned'] = ocr_aligned
test_data['gs'] = gs
test_data['ocr'] = ocr

In [100]:
test_data = test_data.assign(source = lambda df: df.ocr_aligned.str.replace("@", ""))

In [101]:
def levenshtein(reference, hypothesis, progress_bar = False):
    #assert len(reference) == len(hypothesis)
    text = zip(reference, hypothesis)
    if progress_bar:
        text = tqdm(text, total = len(reference))
    d = [distance(r, h) for r, h in text]
    output = pd.DataFrame({"reference":reference, "hypothesis":hypothesis})\
    .assign(distance = lambda df: d)\
    .assign(
        cer = lambda df: df.apply(
            lambda r: 100 * r["distance"] / max(len(r["reference"]), 1), 
            axis = 1
        )
    )
    return output

In [102]:
#arr = test_data["ocr_to_input"].to_numpy()
# gs_arr = test_data["gs_aligned"].to_numpy()
ocr_list = test_data["ocr_aligned"].tolist()
gs_list = test_data["gs_aligned"].tolist()

In [103]:
from math import exp

def uniform(j, window_size):
    return 1.0

def triangle(j, window_size):
    m = window_size//2
    return m - 0.5 * abs(m - j)

def bell(j, window_size):
    m = window_size // 2
    s = window_size // 2
    return exp(-((m-j)/s)**2)

def disjoint(
    string,
    model, 
    source_index,
    target_index,
    window_size = 50,
    decoding_method = "greedy_search", 
    document_progress_bar = False, 
    document_batch_progress_bar = 0, 
    *arcorrect
):
    model.eval()
    windows = [string[i:i+window_size] 
        for i in range(0, len(string), window_size)]
    windows = ["".join([source_index.vocabulary.lookup(c) for c in s])\
        .replace("<UNK>", " ") for s in windows]
    #X = source_index.text2tensor(windows, progress_bar = False).cuda()
    X = source_index.text2tensor(windows, progress_bar = False).cpu()
    if decoding_method == "greedy_search":
        predictions, probs = seq2seq.greedy_search(
            model,
            X, 
            predictions = window_size, 
            progress_bar = document_batch_progress_bar, 
            *arcorrect
        )
    elif decoding_method == "beam_search":
        predictions, probs = seq2seq.beam_search(
            model,
            X, 
            predictions = window_size, 
            progress_bar = document_batch_progress_bar, 
            *arcorrect
        )   
        predictions = predictions[:, 0, :]
    output = target_index.tensor2text(predictions)
    output = [re.sub(r"<START>|<PAD>|<UNK>|<END>.*", "", s) for s in output]
    return "".join(output)

def n_grams(
    string,
    model,
    source_index,
    target_index,
    window_size = 50, 
    decoding_method = "greedy_search", 
    weighting = "uniform",
    document_progress_bar = False, 
    document_batch_progress_bar = 0,      
    main_batch_size = 1024,
    *arcorrect
):
    model.eval()
    if len(string) <= window_size:
        windows = [string]
    else:
        windows = [string[i:i + window_size] 
            for i in range(len(string) - window_size + 1)]
    windows = ["".join([source_index.vocabulary.lookup(c) for c in s])\
    .replace("<UNK>", " ") for s in windows]
    #X = source_index.text2tensor(windows, progress_bar = False).cuda()
    X = source_index.text2tensor(windows, progress_bar = False).cpu()
    if decoding_method == "greedy_search":
        predictions, probs = seq2seq.greedy_search(
            model,
            X, 
            predictions = window_size, 
            progress_bar = False, 
            *arcorrect
        )
    if decoding_method == "beam_search":
        predictions, probs = seq2seq.beam_search(
            model,
            X, 
            predictions = window_size, 
            progress_bar = document_batch_progress_bar, 
            *arcorrect
        )   
        predictions = predictions[:, 0, :]   
    output = target_index.tensor2text(predictions)
    output = [re.sub(r"<START>|<PAD>|<UNK>|<END>.*", "", s) for s in output]
    if weighting == "uniform":
        weighting = uniform
    elif weighting == "triangle":
        weighting = triangle
    elif weighting == "bell":
        weighting = bell
    votes = [
        {k:0.0 for k in target_index.vocabulary} 
        for c in string
    ]
    for i, s in enumerate(output):
        for j, (counter, char)\
        in enumerate(zip(votes[i:i + window_size], s)):
            counter[char] += weighting(j, window_size)
    output = [max(c.keys(), key = lambda x: c[x]) for c in votes]
    output = "".join(output)
    return votes, output


In [114]:
#source = [list("abcdefghijkl"), list("mnopqrstwxyz")]
#target = [list("ABCDEFGHIJKL"), list("MNOPQRSTWXYZ")] #update to astronomy vocab
source = []#yy
for s in model.source_index:
    sentence = ""
    for c in s:
        #nu = source_index[c]
        #sentence.append(c)
        #sentence.append(num)
        sentence += c
    source.append(sentence)

In [132]:
new_source = []
n = 20 # chunk length  
#for s in arr:
for x in range(min([48,ntest])):#only process n paragraphs?
    s = ocr_list[x]
    #sentence_array = list(s) #convert string to an array of characters
    chunks = [s[i:i+n] for i in range(0, len(s), n)]
    for sentence_chunk in chunks:
        new_source.append(list(sentence_chunk))#append array to new_source array

    #new_source.append(list("stars cataclysmic  "))#mark end of sentences with stars cataclysmic
    new_source.append( list("advice charm touch  "))#new end of sentence with 20 chars
    #because the model only allows 20 characters at a time, each row is 20 chars chunks

In [133]:
#new_source

In [134]:
from pytorch_beam_search import seq2seq
source_index = seq2seq.Index(source)
target_index =  source_index
X_new = source_index.text2tensor(new_source)

In [135]:
# predictions, log_probabilities = beam_search(
#     model,
#     X_new.cuda())

# local
predictions, log_probabilities = beam_search(
    model,
    X_new.cpu())

In [136]:
#predictions.cuda()
#predictions.cpu() # local

In [137]:
predictions[0]

tensor([[ 1,  5, 58, 77, 74, 94,  5, 88, 90, 74, 76, 74, 88, 89, 74, 73,  5, 89,
         77, 70, 89],
        [ 1,  5, 58, 77, 74, 91,  5, 88, 90, 74, 76, 74, 88, 89, 74, 73,  5, 89,
         77, 70, 89],
        [ 1,  5, 58, 77, 74, 94,  5, 88, 90, 74, 76, 38, 88, 89, 74, 73,  5, 89,
         77, 70, 89],
        [ 1,  5, 58, 77, 74, 94,  5, 88, 90, 74, 76, 38, 38, 38, 74, 73,  5, 89,
         77, 70, 89],
        [ 1,  5,  5, 58, 77, 74, 94,  5, 88, 90, 74, 76, 74, 88, 89, 74, 73,  5,
         89, 77, 70]])

In [138]:
output = [target_index.tensor2text(p) for p in predictions]
# output = []
# for p in predictions:
#     try:
#         xo = target_index.tensor2text(p)
#         output.append(xo)
#     except:
#         print('key error', np.unique(p))
#         output.append('KEY ERROR')

In [139]:
#output

In [140]:
#output
#output2 = [source_index.tensor2text(p) for p in predictions]

In [141]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
predictions.to(device)

tensor([[[ 1,  5, 58,  ..., 77, 70, 89],
         [ 1,  5, 58,  ..., 77, 70, 89],
         [ 1,  5, 58,  ..., 77, 70, 89],
         [ 1,  5, 58,  ..., 77, 70, 89],
         [ 1,  5,  5,  ..., 89, 77, 70]],

        [[ 1,  5, 84,  ..., 82,  5,  5],
         [ 1,  5, 84,  ..., 82,  5,  5],
         [ 1,  5, 84,  ..., 84, 82,  5],
         [ 1,  5, 84,  ..., 87, 84, 82],
         [ 1,  5, 84,  ..., 84, 82,  5]],

        [[ 1, 77, 74,  ..., 72, 72, 87],
         [ 1, 89, 77,  ..., 70, 72, 72],
         [ 1, 71, 74,  ..., 72, 72, 87],
         [ 1,  5, 89,  ...,  5, 70, 72],
         [ 1, 77, 74,  ..., 70, 72, 72]],

        ...,

        [[ 1, 81,  5,  ...,  5, 76, 70],
         [ 1, 81,  5,  ...,  5, 74, 70],
         [ 1, 81,  5,  ...,  5, 76, 70],
         [ 1, 89,  5,  ...,  5, 76, 70],
         [ 1, 81,  5,  ...,  5,  5, 35]],

        [[ 1, 87, 81,  ...,  2, 46, 14],
         [ 1, 87, 81,  ..., 19,  2,  2],
         [ 1, 87, 81,  ..., 19,  2,  2],
         [ 1, 87, 81,  ...,  2, 23,

In [148]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
predictions.to(device)
use_cuda = True
if device == 'cpu': use_cuda = False
device
#torch.cuda.get_device_name(0)
#torch.cpu.get_device_name(0)

'cpu'

In [150]:
from math import exp

def uniform(j, window_size):
    return 1.0

def triangle(j, window_size):
    m = window_size//2
    return m - 0.5 * abs(m - j)

def bell(j, window_size):
    m = window_size // 2
    s = window_size // 2
    return exp(-((m-j)/s)**2)

def disjoint(
    string,
    model, 
    source_index,
    target_index,
    window_size = 50,
    decoding_method = "greedy_search", 
    document_progress_bar = False, 
    document_batch_progress_bar = 0, 
    use_cuda = True,
    *arcorrect
):
    model.eval()
    windows = [string[i:i+window_size] 
        for i in range(0, len(string), window_size)]
    windows = ["".join([source_index.vocabulary.lookup(c) for c in s])\
        .replace("<UNK>", " ") for s in windows]
    if use_cuda:
        X = source_index.text2tensor(windows, progress_bar = False).cuda()
    else:
        X = source_index.text2tensor(windows, progress_bar = False).cpu()
    if decoding_method == "greedy_search":
        predictions, probs = seq2seq.greedy_search(
            model,
            X, 
            predictions = window_size, 
            progress_bar = document_batch_progress_bar, 
            *arcorrect
        )
    elif decoding_method == "beam_search":
        predictions, probs = seq2seq.beam_search(
            model,
            X, 
            predictions = window_size, 
            progress_bar = document_batch_progress_bar, 
            *arcorrect
        )   
        predictions = predictions[:, 0, :]
    output = target_index.tensor2text(predictions)
    output = [re.sub(r"<START>|<PAD>|<UNK>|<END>.*", "", s) for s in output]
    return "".join(output)

def n_grams(
    string,
    model,
    source_index,
    target_index,
    window_size = 50, 
    decoding_method = "greedy_search", 
    weighting = "uniform",
    document_progress_bar = False, 
    document_batch_progress_bar = 0,      
    main_batch_size = 1024,
    use_cuda=True,
    *arcorrect
):
    model.eval()
    if len(string) <= window_size:
        windows = [string]
    else:
        windows = [string[i:i + window_size] 
            for i in range(len(string) - window_size + 1)]
    windows = ["".join([source_index.vocabulary.lookup(c) for c in s])\
    .replace("<UNK>", " ") for s in windows]
    if use_cuda:
        X = source_index.text2tensor(windows, progress_bar = False).cuda()
    else:
        X = source_index.text2tensor(windows, progress_bar = False).cpu()
    if decoding_method == "greedy_search":
        predictions, probs = seq2seq.greedy_search(
            model,
            X, 
            predictions = window_size, 
            progress_bar = False, 
            *arcorrect
        )
    if decoding_method == "beam_search":
        predictions, probs = seq2seq.beam_search(
            model,
            X, 
            predictions = window_size, 
            progress_bar = document_batch_progress_bar, 
            *arcorrect
        )   
        predictions = predictions[:, 0, :]   
    output = target_index.tensor2text(predictions)
    output = [re.sub(r"<START>|<PAD>|<UNK>|<END>.*", "", s) for s in output]
    if weighting == "uniform":
        weighting = uniform
    elif weighting == "triangle":
        weighting = triangle
    elif weighting == "bell":
        weighting = bell
    votes = [
        {k:0.0 for k in target_index.vocabulary} 
        for c in string
    ]
    for i, s in enumerate(output):
        for j, (counter, char)\
        in enumerate(zip(votes[i:i + window_size], s)):
            counter[char] += weighting(j, window_size)
    output = [max(c.keys(), key = lambda x: c[x]) for c in votes]
    output = "".join(output)
    return votes, output


In [144]:
import time

In [151]:
def full_evaluation(
    raw, 
    gs, 
    model, 
    source_index,
    target_index,
    save_path = None, 
    window_size = 40, 
    document_progress_bar = False, 
    use_cuda = True
):
    print("evaluating all methods...")
    metrics = []
    old = levenshtein(reference = gs, hypothesis = raw).cer.mean()
    # disjoint
    print("  disjoint window...")
    print("    greedy_search...")
    #start = t()
    start = time.time()
    corrections = [
        disjoint(
            s, 
            model, 
            source_index,
            target_index,
            document_progress_bar = document_progress_bar, 
            window_size = window_size,
            use_cuda=use_cuda
        ) for s in raw]
    #print("hello")
    metrics.append(
        {
            "window":"disjoint", 
            "decoding":"greedy",
            "window_size":window_size * 2,
            "weighting":pd.NA,
            #"inference_seconds":t() - start,
            "inference_seconds":time.time() - start,
            "cer_before":old,
            "cer_after":levenshtein(gs, corrections).cer.mean()
        }
    )
    if save_path:
        pd.DataFrame(metrics).assign(
            improvement = lambda df: 100 * (1 - df.cer_after / df.cer_before)
        ).to_csv(save_path, index = False)
    #start = t()
    start = time.time()
    corrections = [
        disjoint(
            s, 
            model, 
            source_index,
            target_index,
            document_progress_bar = document_progress_bar, 
            window_size = window_size * 2,
            use_cuda=use_cuda
        ) 
        for s in raw
    ]
    metrics.append(
        {
            "window":"disjoint", 
            "decoding":"greedy",
            "window_size":window_size,
            "weighting":pd.NA,
            #"inference_seconds":t() - start,
            "inference_seconds":time.time() - start,
            "cer_before":old,
            "cer_after":levenshtein(gs, corrections).cer.mean()
        }
    )
    if save_path:
        pd.DataFrame(metrics).assign(
            improvement = lambda df: 100 * (1 - df.cer_after / df.cer_before)
        ).to_csv(save_path, index = False)
    print("    beam_search...")
    #start = t()
    start = time.time()
    corrections = [
        disjoint(
            s, 
            model, 
            source_index,
            target_index,
            decoding_method = "beam_search", 
            document_progress_bar = document_progress_bar, 
            window_size = window_size,
            use_cuda=use_cuda
        ) 
        for s in raw
    ]
    metrics.append(
        {
            "window":"disjoint", 
            "decoding":"beam", 
            "window_size":window_size * 2,
            "weighting":pd.NA,
            #"inference_seconds":t() - start,
            "inference_seconds":time.time() - start,
            "cer_before":old,
            "cer_after":levenshtein(gs, corrections).cer.mean()
        }
    )
    if save_path:
        pd.DataFrame(metrics).assign(
            improvement = lambda df: 100 * (1 - df.cer_after / df.cer_before)
        ).to_csv(save_path, index = False)
    #start = t()
    start = time.time()
    corrections = [
        disjoint(
            s, 
            model, 
            source_index,
            target_index,
            decoding_method = "beam_search", 
            document_progress_bar = document_progress_bar, 
            window_size = window_size * 2,
            use_cuda=use_cuda
        ) 
        for s in raw
    ]
    metrics.append(
        {
            "window":"disjoint", 
            "decoding":"beam", 
            "window_size":window_size,
            "weighting":pd.NA,
            #"inference_seconds":t() - start,
            "inference_seconds":time.time() - start,
            "cer_before":old,
            "cer_after":levenshtein(gs, corrections).cer.mean()
        }
    )
    if save_path:
        pd.DataFrame(metrics).assign(
            improvement = lambda df: 100 * (1 - df.cer_after / df.cer_before)
        ).to_csv(save_path, index = False)
    # sliding
    print("  sliding")
    print("    greedy...")
    ## greedy search
    print("      uniform...")
    #start = t()
    start = time.time()
    corrections = [
        n_grams(
            s, 
            model, 
            source_index,
            target_index,
            weighting = uniform,
            document_progress_bar = document_progress_bar,
            window_size = window_size,
            use_cuda=use_cuda
        )[1] 
        for s in raw
    ]
    metrics.append(
        {
            "window":"sliding", 
            "decoding":"greedy", 
            "window_size":window_size,
            "weighting":"uniform",
            #"inference_seconds":t() - start,
            "inference_seconds":time.time() - start,
            "cer_before":old,
            "cer_after":levenshtein(gs, corrections).cer.mean()
        }
    )
    if save_path:
        pd.DataFrame(metrics).assign(
            improvement = lambda df: 100 * (1 - df.cer_after / df.cer_before)
        ).to_csv(save_path, index = False)

    print("      triangle...")
    #start = t()
    start = time.time()
    corrections = [
        n_grams(
            s, 
            model, 
            source_index,
            target_index,
            weighting = triangle, 
            document_progress_bar = document_progress_bar,
            window_size = window_size,
            use_cuda=use_cuda
        )[1] 
        for s in raw
    ]
    metrics.append(
        {
            "window":"sliding", 
            "decoding":"greedy", 
            "window_size":window_size,
            "weighting":"triangle",
            #"inference_seconds":t() - start,
            "inference_seconds":time.time() - start,
            "cer_before":old,
            "cer_after":levenshtein(gs, corrections).cer.mean()
        }
    )
    if save_path:
        pd.DataFrame(metrics).assign(
            improvement = lambda df: 100 * (1 - df.cer_after / df.cer_before)
        ).to_csv(save_path, index = False)

    print("      bell...")
    #start = t()
    start = time.time()
    corrections = [
        n_grams(
            s, 
            model, 
            source_index,
            target_index,
            weighting = bell, 
            document_progress_bar = document_progress_bar,
            window_size = window_size,
            use_cuda=use_cuda
        )[1] 
        for s in raw
    ]
    metrics.append(
        {
            "window":"sliding", 
            "decoding":"greedy", 
            "window_size":window_size,
            "weighting":"bell",
            #"inference_seconds":t() - start,
            "inference_seconds":time.time() - start,
            "cer_before":old,
            "cer_after":levenshtein(gs, corrections).cer.mean()
        }
    )
    if save_path:
        pd.DataFrame(metrics).assign(
            improvement = lambda df: 100 * (1 - df.cer_after / df.cer_before)
        ).to_csv(save_path, index = False)

    ## beam search
    print("    beam...")
    print("      uniform...")
    #start = t()
    start = time.time()
    corrections = [
        n_grams(
            s, 
            model, 
            source_index,
            target_index,
            decoding_method = "beam_search", 
            weighting = uniform, 
            document_progress_bar = document_progress_bar,
            window_size = window_size,
            use_cuda=use_cuda
        )[1] 
        for s in raw
    ]
    metrics.append(
        {
            "window":"sliding", 
            "decoding":"beam", 
            "window_size":window_size,
            "weighting":"uniform",
            #"inference_seconds":t() - start,
            "inference_seconds":time.time() - start,
            "cer_before":old,
            "cer_after":levenshtein(gs, corrections).cer.mean()
        }
    )
    if save_path:
        pd.DataFrame(metrics).assign(
            improvement = lambda df: 100 * (1 - df.cer_after / df.cer_before)
        ).to_csv(save_path, index = False)

    print("      triangle...")
    #start = t()
    start = time.time()
    corrections = [
        n_grams(
            s, 
            model, 
            source_index,
            target_index,
            decoding_method = "beam_search", 
            weighting = triangle, 
            document_progress_bar = document_progress_bar,
            window_size = window_size,
            use_cuda=use_cuda
        )[1] 
        for s in raw
    ]
    metrics.append(
        {
            "window":"sliding", 
            "decoding":"beam", 
            "window_size":window_size,
            "weighting":"triangle",
            #"inference_seconds":t() - start,
            "inference_seconds":time.time() - start,
            "cer_before":old,
            "cer_after":levenshtein(gs, corrections).cer.mean()
        }
    )
    if save_path:
        pd.DataFrame(metrics).assign(
            improvement = lambda df: 100 * (1 - df.cer_after / df.cer_before)
        ).to_csv(save_path, index = False)

    print("      bell...")
    #start = t()
    start = time.time()
    if use_cuda:
        corrections = [
            n_grams(
                s, 
                model.cuda(), 
                source_index,
                target_index,
                decoding_method = "beam_search", 
                weighting = bell, 
                document_progress_bar = document_progress_bar,
                window_size = window_size,
                use_cuda=use_cuda
            )[1] 
            for s in raw
        ]
    else:
        corrections = [
            n_grams(
                s, 
                model.cpu(), 
                source_index,
                target_index,
                decoding_method = "beam_search", 
                weighting = bell, 
                document_progress_bar = document_progress_bar,
                window_size = window_size,
                use_cuda=use_cuda
            )[1] 
            for s in raw
        ]
    metrics.append(
        {
            "window":"sliding", 
            "decoding":"beam", 
            "window_size":window_size,
            "weighting":"bell",
            #"inference_seconds":t() - start,
            "inference_seconds":time.time() - start,
            "cer_before":old,
            "cer_after":levenshtein(gs, corrections).cer.mean()
        }
    )
    if save_path:
        pd.DataFrame(metrics).assign(
            improvement = lambda df: 100 * (1 - df.cer_after / df.cer_before)
        ).to_csv(save_path, index = False)
    print()
    return pd.DataFrame(metrics).assign(
        improvement = lambda df: 100 * (1 - df.cer_after / df.cer_before)
    )

In [152]:
#evaluation
#source_index

In [ ]:
print("\nevaluating all correction methods...")
evaluation = full_evaluation(
    ocr_list,
    gs_list,
    model,
    source_index,
    target_index,
    use_cuda=use_cuda
)

print("\n--results--")
print("test data:", test)
print("plain beam search:", just_beam)
print("disjoint")
print("  greedy search:", disjoint_greedy)
print("  beam search:", disjoint_beam)
print("n_grams")
print("  greedy search:", n_grams_greedy)
print("  beam search:", n_grams_beam)

print("\n", evaluation)


evaluating all correction methods...
evaluating all methods...
  disjoint window...
    greedy_search...
    beam_search...


In [ ]:
gs_list #take out @

In [ ]:
evaluation

In [ ]:
uniform

In [ ]:
output_folder

In [ ]:
evaluation.to_csv(output_folder + "eval/evaluation60new.csv")

In [ ]:
new_output = []
sentence = ""
for chunks in output:
    #if chunks[0] == 'stars cataclysmic  ':#everytime it finds stars cataclysmic it outputs a new sentence #theres only 19?
    if chunks[0] == '<START>advice charm touch  ':#everytime it finds advice charm touch   it outputs a new sentence
        new_output.append(sentence)
        sentence = ""# this empties the sentence variable whenever a sentence is added to the array
    else:
       new_sentence = chunks[0].replace("<START>", "")
       new_sentence = new_sentence.replace("<END>", "")
       sentence = sentence + new_sentence #combining the chunks of 20 characters together without the start and end
        

In [ ]:
#new_output

In [ ]:
import numpy as np
import pandas as pd
unnamed = list(range(1,49))
#print(unnamed)
test_data['new_output'] = new_output
test_data['unnamed'] = unnamed

In [ ]:
#test_data
#test_data.to_csv('./github_output_df.csv')

In [ ]:
# !pip install pybind11
# !pip install fastwer
#!pip install pytesseract
#!sudo apt install tesseract-ocr

import cv2
#import pytesseract
import fastwer
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image, display

In [ ]:
for index, row in test_data.iterrows():
  filename = row['unnamed']
  ref = row['gs_aligned']
  ocr = row['ocr_aligned']
  output = row['new_output']
  cer = fastwer.score_sent(ocr, ref, char_level=True)
  wer = fastwer.score_sent(ocr, ref, char_level=False)
  test_data.loc[test_data['unnamed'] == filename, 'before_cer'] = round(cer,2) # Round value to 2 decimal places
  test_data.loc[test_data['unnamed'] == filename, 'before_wer'] = round(wer,2)

#test_data

In [ ]:
test_data.to_csv(output_dir + 'eval/cer_wer_new50.csv')

In [ ]:
for index, row in test_data.iterrows():
  filename = row['unnamed']
  ref = row['gs_aligned']
  ocr = row['ocr_aligned']
  output = row['new_output']
  cer = fastwer.score_sent(output, ref, char_level=True)
  wer = fastwer.score_sent(output, ref, char_level=False)
  test_data.loc[test_data['unnamed'] == filename, 'after_cer'] = round(cer,2) # Round value to 2 decimal places
  test_data.loc[test_data['unnamed'] == filename, 'after_wer'] = round(wer,2)

#test_data

In [ ]:
test_data.to_csv('./cer_wer_new50epochs.csv')

In [ ]:
#test_data

In [ ]:
#output

In [ ]:
sentence

In [ ]:
from nltk import tokenize
new_output = tokenize.sent_tokenize(sentence)
#new_output

In [ ]:
gs =test_data['gs_aligned']
gs_list = ' '.join(test_data['gs_aligned'].tolist())
#gs_array = gs.to_numpy()

#print(gs_array)
new_gs = tokenize.sent_tokenize(gs_list) # trying to take gs column to match the length of the new_output.
len(new_gs)

In [ ]:
def readTest():   
    input_file = './post_ocr_correction/data/en/data/test'
    
    # reverse is true, therefore pair[0] is ocr and pair[1] is gs
    input_lang, output_lang, pairs = prepareData(input_file,'gs', 'ocr', True)
    
    output_sentences = []
    input_sentences = []
    corrected_sentences = []
    for pair in pairs:
        sentence = pair[0]  #ocr sentence
        try:
            output_words, attentions = evaluate(encoder1, attn_decoder1, sentence)
            input_sentences.append(pair[1])  #gs sentence
            output_sentences.append(pair[0]) #ocr sentence
            corrected_sentences.append(' '.join(output_words))
 
            #testing_df.append({'output':''.join(output_words),'input':x},ignore_index=True)
            #showAttention(x, output_words, attentions)
        except KeyError:
            print('KeyError =', sentence)
    testing_df = pd.DataFrame({"gs":input_sentences, "ocr":output_sentences, "output":corrected_sentences})
    return testing_df
testing_dataframe1 = readTest()
testing_dataframe1

In [ ]:
import pandas as pd
output_df1 = pd.DataFrame({"output":new_output})
output_df1.to_csv('./output_dataframe_new.csv')

In [ ]:
output_df1

In [ ]:
def readTest():   
    input_file = './post_ocr_correction/data/en/data/test'
    
    # reverse is true, therefore pair[0] is ocr and pair[1] is gs
    input_lang, output_lang, pairs = prepareData(input_file,'gs', 'ocr', True)
    
    output_sentences = []
    input_sentences = []
    corrected_sentences = []
    for pair in pairs:
        sentence = pair[0]  #ocr sentence
        try:
            output_words, attentions = evaluate(encoder1, attn_decoder1, sentence)
            input_sentences.append(pair[1])  #gs sentence
            output_sentences.append(pair[0]) #ocr sentence
            corrected_sentences.append(' '.join(output_words))
 
            #testing_df.append({'output':''.join(output_words),'input':x},ignore_index=True)
            #showAttention(x, output_words, attentions)
        except KeyError:
            print('KeyError =', sentence)
    testing_df = pd.DataFrame({"gs":input_sentences, "ocr":output_sentences, "output":corrected_sentences})
    return testing_df
testing_dataframe1 = readTest()
testing_dataframe1

In [ ]:
# import pandas as pd
# #dictionary = {'GS': "", 'OCR': "", 'index'=[0]} 
# #training_df = pd.DataFrame(dictionary)

# ocr =[]
# ocr_aligned=[]
# gs_aligned=[]

# for x in range(1008,1120):
#     for y in range(len(PDF_OUT_SENT[x])):
#         for z in range(len(PDF_OUT_SENT[x][y])):
#             OCR = OCR_OUT_SENT[x][y][z]
#             OCR_aligned_train = OCR_aligned_SENT[x][y][z]
#             GS_aligned_train = PDF_OUT_SENT[x][y][z]
#             #dictionary = {'GS': GS_train, 'OCR': OCR_train} 
#             #training_df = pd.DataFrame(dictionary)
#             #gs_train = training_df.append(OCR_OUT_SENT)
#             ocr.append(OCR)
#             ocr_aligned.append(OCR_aligned_train)
#             gs_aligned.append(GS_aligned_train)
#             #training_df = pd.concat([training_df, pd.DataFrame(dictionary)], ignore_index = True)
# data_ocr = pd.DataFrame({"ocr_to_input":ocr}) #"ocr_aligned":ocr_aligned, "gs_aligned":gs_aligned})
# print(data_ocr.shape)
# data_ocr

In [ ]:
idx, probs = model.predict(dev_source)

In [ ]:
model.tensor2text(idx)

In [ ]:
model.tensor2text(train_target[:10])

In [ ]:
model.tensor2text(train_source[:10])